In [2]:
import anndata as ad
import matplotlib.pyplot as plt
import mudata as md
import muon
import scanpy as sc
from scvi.data import pbmc_seurat_v4_cite_seq
from scvi.model import TOTALVI
import sys
import numpy as np
import pickle
from copy import deepcopy
import numpy as np
import pandas as pd

import cosg as cosg
import importlib
import time
importlib.reload(cosg)

/home/bram/miniconda3/envs/joc/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/bram/miniconda3/envs/joc/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


<module 'cosg' from '/home/bram/miniconda3/envs/joc/lib/python3.10/site-packages/cosg/__init__.py'>

In [3]:
dataLoc = './data/'
adata = pbmc_seurat_v4_cite_seq(save_path=dataLoc, apply_filters=True, aggregate_proteins=True, mask_protein_batches=0)

adata.layers["counts"] = adata.X.copy()
# sc.pp.normalize_total(adata)
# sc.pp.log1p(adata)
adata.obs_names_make_unique()

INFO     File ./data/pbmc_seurat_v4.h5ad already downloaded                                                        


In [4]:
# Remove the CD14 Mono cells
# adata = adata[adata.obs['celltype.l2'] != 'CD14 Mono', :]
adata

AnnData object with n_obs × n_vars = 152094 × 20729
    obs: 'nCount_ADT', 'nFeature_ADT', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'lane', 'donor', 'time', 'celltype.l1', 'celltype.l2', 'celltype.l3', 'Phase', 'nCount_SCT', 'nFeature_SCT', 'X_index', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'Protein log library size', 'Number proteins detected', 'RNA log library size'
    var: 'mt'
    obsm: 'protein_counts'
    layers: 'counts'

In [5]:
protein_adata = ad.AnnData(adata.obsm["protein_counts"])
protein_adata.obs_names = adata.obs_names
del adata.obsm["protein_counts"]


In [6]:
mdata = md.MuData({"rna": adata, "protein": protein_adata})
mdata.var_names_make_unique()

/home/bram/miniconda3/envs/joc/lib/python3.10/site-packages/mudata/_core/mudata.py:491: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(
/home/bram/miniconda3/envs/joc/lib/python3.10/site-packages/mudata/_core/mudata.py:1059: UserWarning: Modality names will be prepended to var_names since there are identical var_names in different modalities.
  warnings.warn(


In [7]:
sc.pp.highly_variable_genes(
    mdata.mod["rna"],
    n_top_genes=5000,
    flavor="seurat_v3",
#    batch_key="batch",
    layer="counts")

: 

In [9]:
markers = set()
with open('data/cosg_30_markergenes.csv') as f:
    for i, line in enumerate(f):
        if i > 0:
            genes = line.split(',')[1:]
            genes = ['rna:' + g.rstrip('\n') for g in genes]

            markers = markers.union(set(genes))

markers = sorted(list(markers))
len(markers)

819

In [ ]:
# Remove all marker genes from the rna_subset
# mdata.mod['rna_subset'] = mdata.mod['rna'][:, markers].copy()


In [ ]:
all_genes = mdata['rna'].var.index.values
print(len(all_genes))



20729


In [ ]:
# Get the difference between all_genes and markers
non_markers = np.setdiff1d(all_genes, markers)
print(len(non_markers))

19910


In [ ]:
print(mdata.mod['rna_subset'][:, :])

View of AnnData object with n_obs × n_vars = 152094 × 819
    obs: 'nCount_ADT', 'nFeature_ADT', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'lane', 'donor', 'time', 'celltype.l1', 'celltype.l2', 'celltype.l3', 'Phase', 'nCount_SCT', 'nFeature_SCT', 'X_index', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'Protein log library size', 'Number proteins detected', 'RNA log library size'
    var: 'mt'
    layers: 'counts'
